In [1]:
# imports
import pandas as pd
from keras import models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

2022-11-08 16:15:12.848107: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# load dataset
df = pd.read_csv('/Users/taylor/Desktop/DS340W/term_project/data/wisconsin/wisconsin_train_balanced.csv')
dataset = df.values

# split into input (X) and output (Y) variables
X = dataset[:,0:30].astype(float)
Y = dataset[:,30]

# encode class values as integers
encoder = LabelEncoder()

encoder.fit(Y)
encoded_Y = encoder.transform(Y)

In [3]:
# baseline model
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(30, input_shape=(30,), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [4]:
# evaluate baseline model with standardized dataset
estimator = KerasClassifier(model=create_baseline, epochs=100, batch_size=32, verbose=0) # baseline epochs chosen by 3 * ncol
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, encoded_Y, cv=kfold)

print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

2022-11-08 16:15:18.815369: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Baseline: 98.56% (1.08%)


In [5]:
# Repeat w/ smaller model:

def create_smaller():
    # create model
    model = Sequential()
    model.add(Dense(15, input_shape=(30,), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [6]:
# evaluate smaller model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(model=create_smaller, epochs=100, batch_size=32, verbose=0)))

pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)

print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Smaller: 100.00% (0.00%)


In [7]:
# Repeat w/ larger model (i.e., 10, ..., 13 hidden layers):

# larger model
def create_larger():
    # create model
    model = Sequential()
    
    model.add(Dense(30, input_shape=(30,), activation='relu'))
    model.add(Dense(15, activation='relu'))
    model.add(Dense(15, activation='relu'))
    model.add(Dense(15, activation='relu'))
    model.add(Dense(15, activation='relu'))
    model.add(Dense(15, activation='relu'))
    model.add(Dense(15, activation='relu'))
    model.add(Dense(15, activation='relu'))
    model.add(Dense(15, activation='relu'))
    model.add(Dense(15, activation='relu'))
    model.add(Dense(15, activation='relu'))
    model.add(Dense(15, activation='relu'))
    
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [8]:
# evaluate larger model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(model=create_larger, epochs=100, batch_size=32, verbose=0)))

pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)

print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Larger: 99.79% (0.62%)


In [9]:
# save models
model1 = create_baseline()
model2 = create_smaller()
model3 = create_larger()

model1.save('/Users/taylor/Desktop/DS340W/term_project/models/baseline')
model2.save('/Users/taylor/Desktop/DS340W/term_project/models/smaller')
model3.save('/Users/taylor/Desktop/DS340W/term_project/models/12layerNN')

INFO:tensorflow:Assets written to: /Users/taylor/Desktop/DS340W/term_project/models/baseline/assets
INFO:tensorflow:Assets written to: /Users/taylor/Desktop/DS340W/term_project/models/smaller/assets
INFO:tensorflow:Assets written to: /Users/taylor/Desktop/DS340W/term_project/models/12layerNN/assets


In [11]:
df.columns

Index(['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean',
       'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave.points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave.points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave.points_worst',
       'symmetry_worst', 'fractal_dimension_worst', 'diagnosis'],
      dtype='object')